In [ ]:
import os
import sys
from pathlib import Path

import ipywidgets as widgets

# Add the mlops parent directory to the PYTHONPATH
sys.path.insert(0, os.path.abspath(".."))

from utils.key_pair import read_public_key_from_pem, write_private_key_to_pem

%load_ext autoreload
%autoreload 2

In [ ]:
DEFAULT_PEM_PATH = "~/.ssh/magic-packet-demand-instance.pem"
pem_path = ""
public_key = ""

# (Optional) Create an RSA key pair

The key pair is used to SSH into the EC2 instance. For more on Amazon EC2 key pairs see [Amazon EC2 key pairs and Linux instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html).

In [ ]:
pem_path = (
    input(f"Enter the PEM path to write the RSA private key (or {DEFAULT_PEM_PATH}): ")
    or DEFAULT_PEM_PATH
)
write_private_key_to_pem(pem_path, input('Enter the passhprase (or ""): '))

Make sure to set the proper permissions on the key file ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html))

In [ ]:
!chmod 400 $pem_path

# Specify terraform input variables

Inputs to read the public key from the PEM containing the private key

In [ ]:
pem_path = (
    input(
        f"Enter the PEM path containing the RSA private key (or {DEFAULT_PEM_PATH}): "
    )
    or DEFAULT_PEM_PATH
)
password = input('Enter the password (or ""): ') or None
public_key = read_public_key_from_pem(pem_path, password)
public_key

Writing a `.tvars` file for easier invocation of the terraform commands. 

If looking for an AMI on the marketplace then follow the instructions in [Find a Linux AMI: Find a Linux AMI using the Amazon EC2 console ](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/finding-an-ami.html). In the search bar for AMIs type "deep learning" to filter to instances with packages pre-installed. The AMI image ID should be viewable in the search results. Make sure to be in the desired region in the console as the IDs change per region. Alternatively use one of the AMIs listed in [Deep Learning AMI: AMI Options](https://docs.aws.amazon.com/dlami/latest/devguide/ami-options.html).

Spot instance pricing may be found in [Amazon EC2 Spot Instances Pricing](https://aws.amazon.com/ec2/spot/pricing/).

In [ ]:
key_name = Path(pem_path).stem
template_tfvars = f"""ami               = "ami-0279c3b3186e54acd"
availability_zone = "us-east-1a"
instance_type     = "t2.micro"
key_name          = "{key_name}"
public_key        = "{public_key}"
region            = "us-east-1"
tag_name          = "mpdi"
"""


def write_tfvars(value):
    with open("terraform.tfvars", "w") as tfvars:
        tfvars.write(value)


write_tfvars(template_tfvars)

# Displaying textarea widget for editting of the .tfvars
layout = widgets.Layout(width="100%", height="100px")
text_area = widgets.Textarea(
    value=template_tfvars, description="tfvars:", continuous_update=False, layout=layout
)
text_area.observe(lambda change: write_tfvars(change["new"]), names="value")
text_area

# Terraform CLI

The `init` command is necessary once before you can run any other commands.

In [ ]:
!terraform init

The `plan` subcommand is useful to sanity check what infrastructure terraform will modify in `apply`

In [ ]:
!terraform plan

`apply` will initiate the spot instance request. Afterwards, if the request is accepted by AWS, an EC2 instance will be accessible over SSH. For more on spot instances see [Spot Instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-spot-instances.html).

Once the EC2 instance is running it may connected to over SSH ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html)):

```sh
ssh -i /path/my-key-pair.pem my-instance-user-name@my-instance-public-dns-name
```

In [ ]:
!terraform apply -auto-approve

Once finished with the EC2 instance it is important to run the `destroy` subcommand to cancel the spot instance request. Otherwise, the request may stay open and AWS will try and fulfill it.

In [ ]:
!terraform destroy -auto-approve